In [1]:
!git clone https://github.com/Anti-Entrophic/FastSpeechPPG.git
%cd /content/FastSpeechPPG/FastSpeech2
!pip3 install -r requirements.txt
!pip install g2p_en
!pip install inflect
!pip install librosa
!pip install matplotlib
!pip install pyworld
!pip install tgt
!pip install pypinyin==0.39.0
!pip install unidecode
!pip install --upgrade --no-cache-dir gdown

Cloning into 'FastSpeechPPG'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 164 (delta 21), reused 71 (delta 7), pack-reused 52
Receiving objects: 100% (164/164), 8.21 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/FastSpeechPPG/FastSpeech2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 99.1 MB/s eta 0:00:00
  Inst

# Infer

In [ ]:
#貌似不行
!gdown "https://drive.google.com/uc?id=1DBRkALpPd6FL9gjHMmMEdHODmkgNIIK4"

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1DBRkALpPd6FL9gjHMmMEdHODmkgNIIK4 



In [ ]:
!python3 synthesize.py --text "大家好" --speaker_id 1 --restore_step 600000 --mode single -p config/AISHELL3/preprocess.yaml -m config/AISHELL3/model.yaml -t config/AISHELL3/train.yaml

Traceback (most recent call last):
  File "/content/FastSpeech2/synthesize.py", line 188, in <module>
    model = get_model(args, configs, device, train=False)
  File "/content/FastSpeech2/utils/model.py", line 20, in get_model
    ckpt = torch.load(ckpt_path)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 771, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 270, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 251, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: './output/ckpt/AISHELL3/600000.pth.tar'


# Train(原版)

## 准备数据

In [ ]:
!pip install wget
!pip install tarfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=bfb823a1a3b5092d0355256a771bdd4daf363f5b22ab19ac0eb09ac3080dee0b
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tarfile (from versions: none)
ERROR: No matching distribution found for tarfile


In [ ]:
#@title 下载 LJspeech
# 3min
%cd /content
!wget --no-check-certificate -r http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2

/content
--2023-03-17 15:40:47--  http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2’

data.keithito.com/d 100%[===================>]   2.56G  12.1MB/s    in 3m 23s  

2023-03-17 15:44:11 (12.9 MB/s) - ‘data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]

FINISHED --2023-03-17 15:44:11--
Total wall clock time: 3m 24s
Downloaded: 1 files, 2.6G in 3m 23s (12.9 MB/s)


In [ ]:
#@title 解压
# 10min
%cd /content/data.keithito.com/data/speech/
import tarfile
filename = "LJSpeech-1.1.tar.bz2"
tf = tarfile.open(filename)
tf.extractall('/content')

## 训练

In [ ]:
#@title 测试日志模块（不用再跑了）
%cd /content
import logging
logging.basicConfig(level=logging.DEBUG #设置日志输出格式
      ,filename="/content/experiment.log" #log日志输出的文件位置和文件名
      ,format="%(asctime)s-%(levelname)s: %(message)s" #日志输出的格式
                      # -8表示占位符，让输出左对齐，输出长度都为8位
      ,datefmt="%Y-%m-%d %H:%M:%S"  #时间输出的格式
      ,force=True)
# test
logging.debug("debug!")

# 这样用就可以了
from debugging import logging_init
import logging

/content


In [ ]:
#@title 预处理，好像是去除文字的大小写和特殊符号。还有对音频处理了一下，看不懂。
# 4min
%cd /content/FastSpeechPPG/FastSpeech2
!python3 prepare_align.py config/LJSpeech/preprocess.yaml

/content/FastSpeechPPG/FastSpeech2
13100it [03:33, 61.28it/s]


上传 `LJSpeech.zip` 到路径 `FastSpeech2/preprocessed_data/LJSpeech/TextGrid/` 下，并解压

In [ ]:
# 解压
%cd /content/FastSpeechPPG/FastSpeech2/preprocessed_data/LJSpeech
import zipfile
import os
assert os.path.exists('./LJSpeech.zip')
zip_file = zipfile.ZipFile('./LJSpeech.zip')
zip_file.extractall()

False
/content/FastSpeechPPG/FastSpeech2/preprocessed_data/LJSpeech


In [15]:
%cd /content/FastSpeechPPG/FastSpeech2
!python3 preprocess.py config/LJSpeech/preprocess.yaml

/content/FastSpeechPPG/FastSpeech2
/content/FastSpeechPPG/FastSpeech2/audio/stft.py:42: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/content/FastSpeechPPG/FastSpeech2/audio/stft.py:145: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0, fmax=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(
Processing Data ...
Traceback (most recent call last):
  File "/content/FastSpeechPPG/FastSpeech2/preprocess.py", line 15, in <module>
    preprocessor.build_from_path()
  File "/content/FastSpeechPPG/FastSpeech2/preprocessor/preprocessor.py", line 67, in build_from_path
    for i, speaker in enumerate(tqdm(os.listdir(self.in_dir))):
FileNotFoundError: [Errno 2] No such file or directory: './raw_data/LJSpeech'


# Train

In [2]:
#@title 下载 librispeech
%cd /content
!wget --no-check-certificate -r https://us.openslr.org/resources/12/train-clean-100.tar.gz

/content
--2023-03-17 23:15:48--  https://us.openslr.org/resources/12/train-clean-100.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6387309499 (5.9G) [application/x-gzip]
Saving to: ‘us.openslr.org/resources/12/train-clean-100.tar.gz’

us.openslr.org/reso 100%[===================>]   5.95G  13.1MB/s    in 8m 11s  

2023-03-17 23:24:00 (12.4 MB/s) - ‘us.openslr.org/resources/12/train-clean-100.tar.gz’ saved [6387309499/6387309499]

FINISHED --2023-03-17 23:24:00--
Total wall clock time: 8m 12s
Downloaded: 1 files, 5.9G in 8m 11s (12.4 MB/s)


In [3]:
#@title 解压 librispeech
%cd /content/us.openslr.org/resources/12
import tarfile
filename = "train-clean-100.tar.gz"
tf = tarfile.open(filename)
tf.extractall('/content')

/content/us.openslr.org/resources/12


In [11]:
!pip install librosa==0.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0


In [5]:
!pip install ffmpeg
!pip install soundfile
import librosa
import soundfile
import glob
import os

src = os.path.join('/content/LibriSpeech/train-clean-100/8629/261139/', '*.flac')
dst = os.path.join('/content/LibriSpeech/train-clean-100/8629/261139/', '*.wav')
filelist = glob.glob(r'/content/LibriSpeech/train-clean-100/8629/261139/*.flac')
for file in filelist:
  src = os.path.join('/content/LibriSpeech/train-clean-100/8629/261139/', file)
  y,sr = librosa.load(src)
  name = os.path.splitext(file)[0] + '.wav'
  dst = os.path.join('/content/LibriSpeech/train-clean-100/8629/261139/', name)
  soundfile.write(dst,y,sr)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
#@title Train
%cd /content/FastSpeechPPG/FastSpeech2
!python3 train.py -p config/LJSpeech/preprocess.yaml -m config/LJSpeech/model.yaml -t config/LJSpeech/train.yaml

/content/FastSpeechPPG/FastSpeech2
Prepare training ...
/content/FastSpeechPPG/FastSpeech2/stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/content/FastSpeechPPG/FastSpeech2/layers.py:50: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0.0, fmax=8000.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(
Number of FastSpeech2 Parameters: 21895353
Removing weight norm...
2023-03-18 01:43:26.391797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 01:43:27.343832: W tensorflow/compiler/xla/str